In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/web-traffic-time-series-forecasting/sample_submission_2.csv.zip
/kaggle/input/web-traffic-time-series-forecasting/sample_submission_1.csv.zip
/kaggle/input/web-traffic-time-series-forecasting/train_2.csv.zip
/kaggle/input/web-traffic-time-series-forecasting/key_1.csv.zip
/kaggle/input/web-traffic-time-series-forecasting/key_2.csv.zip
/kaggle/input/web-traffic-time-series-forecasting/train_1.csv.zip


In [2]:
import numpy as np 
import pandas as pd 
import re
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm
from tqdm import tnrange
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
import datetime
from keras.layers import Dropout
from keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense,Input,LSTM,Flatten,Embedding,Concatenate

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
data = pd.read_csv("/kaggle/input/web-traffic-time-series-forecasting/train_2.csv.zip", compression = "zip")
data.head()

,Page,2015-07-01,2015-07-02,2015-07-03,2015-07-04,2015-07-05,2015-07-06,2015-07-07,2015-07-08,2015-07-09,...,2017-09-01,2017-09-02,2017-09-03,2017-09-04,2017-09-05,2017-09-06,2017-09-07,2017-09-08,2017-09-09,2017-09-10
0,2NE1_zh.wikipedia.org_all-access_spider,18.0,11.0,5.0,13.0,14.0,9.0,9.0,22.0,26.0,...,19.0,33.0,33.0,18.0,16.0,27.0,29.0,23.0,54.0,38.0
1,2PM_zh.wikipedia.org_all-access_spider,11.0,14.0,15.0,18.0,11.0,13.0,22.0,11.0,10.0,...,32.0,30.0,11.0,19.0,54.0,25.0,26.0,23.0,13.0,81.0
2,3C_zh.wikipedia.org_all-access_spider,1.0,0.0,1.0,1.0,0.0,4.0,0.0,3.0,4.0,...,6.0,6.0,7.0,2.0,4.0,7.0,3.0,4.0,7.0,6.0
3,4minute_zh.wikipedia.org_all-access_spider,35.0,13.0,10.0,94.0,4.0,26.0,14.0,9.0,11.0,...,7.0,19.0,19.0,9.0,6.0,16.0,19.0,30.0,38.0,4.0
4,52_Hz_I_Love_You_zh.wikipedia.org_all-access_s...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,16.0,16.0,19.0,9.0,20.0,23.0,28.0,14.0,8.0,7.0


In [4]:
Page=data['Page']
data=data.drop(columns=['Page']).interpolate(axis=1)
data.fillna(0,inplace=True)
data.insert(loc=0,column='Page',value=Page)
data.head()

,Page,2015-07-01,2015-07-02,2015-07-03,2015-07-04,2015-07-05,2015-07-06,2015-07-07,2015-07-08,2015-07-09,...,2017-09-01,2017-09-02,2017-09-03,2017-09-04,2017-09-05,2017-09-06,2017-09-07,2017-09-08,2017-09-09,2017-09-10
0,2NE1_zh.wikipedia.org_all-access_spider,18.0,11.0,5.0,13.0,14.0,9.0,9.0,22.0,26.0,...,19.0,33.0,33.0,18.0,16.0,27.0,29.0,23.0,54.0,38.0
1,2PM_zh.wikipedia.org_all-access_spider,11.0,14.0,15.0,18.0,11.0,13.0,22.0,11.0,10.0,...,32.0,30.0,11.0,19.0,54.0,25.0,26.0,23.0,13.0,81.0
2,3C_zh.wikipedia.org_all-access_spider,1.0,0.0,1.0,1.0,0.0,4.0,0.0,3.0,4.0,...,6.0,6.0,7.0,2.0,4.0,7.0,3.0,4.0,7.0,6.0
3,4minute_zh.wikipedia.org_all-access_spider,35.0,13.0,10.0,94.0,4.0,26.0,14.0,9.0,11.0,...,7.0,19.0,19.0,9.0,6.0,16.0,19.0,30.0,38.0,4.0
4,52_Hz_I_Love_You_zh.wikipedia.org_all-access_s...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,16.0,16.0,19.0,9.0,20.0,23.0,28.0,14.0,8.0,7.0


In [5]:
indices = np.random.permutation(data.shape[0]) 
idx=indices[:int(0.70*data.shape[0])]

In [6]:
x_train=[]
y_train=[]
project=[] 
spider=[] 
access=[] 
for i in tqdm(idx):
    k=data.iloc[i].values[-364:] # Taking last 364 days of data
    x_train.append(k[:300]) # We will use 300 days of data for training and rest 64 days as our target values.
    y_train.append(k[300:])

    projects = ['zh.wikipedia.org', 'fr.wikipedia.org', 'en.wikipedia.org', 'commons.wikimedia.org', 'ru.wikipedia.org', 'www.mediawiki.org', 'de.wikipedia.org', 'ja.wikipedia.org', 'es.wikipedia.org']
    for p in projects:
        if p in Page[i]:
            project.append(p)
            break

    if('spider' in Page[i]):
        spider.append('spider')
    else:
        spider.append('non-spider') 
 
    k=max([i.start() for i in re.finditer('org_',Page[i])])  
    if('all-access' in Page[i][k:]):
        access.append('all_access')
    if('desktop' in Page[i][k:]):
        access.append('desktop')
    if('mobile' in Page[i][k:]): 
        access.append('mobile')   

100%|██████████| 101544/101544 [01:14<00:00, 1369.24it/s]


In [7]:
X=np.array(x_train).reshape(len(idx),300,1)
y=np.array(y_train).reshape(len(idx),64)

In [8]:
X.shape

(101544, 300, 1)

In [9]:
X_train=X[:100000]
y_train=y[:100000]
X_test=X[100000:]
y_test=y[100000:]

In [10]:
prj_enc=LabelEncoder()
prj=prj_enc.fit_transform(project)
access_enc=LabelEncoder()
access=access_enc.fit_transform(access)
spider_enc=LabelEncoder()
spider=spider_enc.fit_transform(spider)

In [11]:
prj_train=prj[:100000]
prj_test=prj[100000:]
access_train=access[:100000] 
access_test=access[100000:]
spider_train=spider[:100000]
spider_test=spider[100000:]

In [12]:
access_train=access_train.reshape(len(access_train),1)
access_test=access_test.reshape(len(access_test),1)
prj_train=prj_train.reshape(len(prj_train),1)
prj_test=prj_test.reshape(len(prj_test),1)
spider_train=spider_train.reshape(len(spider_train),1)
spider_test=spider_test.reshape(len(spider_test),1)

# LSTM

In [13]:
from keras import backend as K
def smape(y_true, y_pred):
    return K.mean(100*K.abs(y_pred - y_true) / (K.abs(y_pred) + K.abs(y_true)), axis=-1)

In [14]:
# Define model architecture
input_layer = Input(shape=(X_train.shape[1], X_train.shape[2]))
x = LSTM(100, return_sequences=True)(input_layer)
x = LSTM(50)(x)

input_layer1 = Input(shape=(access_train.shape[1],))
x1 = Embedding(input_dim=3, output_dim=3)(input_layer1)
x1 = Flatten()(x1)

input_layer2 = Input(shape=(prj_train.shape[1],))
x2 = Embedding(input_dim=8, output_dim=8)(input_layer2)
x2 = Flatten()(x2)

input_layer3 = Input(shape=(spider_train.shape[1],))
x3 = Embedding(input_dim=2, output_dim=2)(input_layer3)
x3 = Flatten()(x3)

concat = Concatenate()([x, x1, x2, x3])
x = Dense(264, activation='relu')(concat)
x = Dropout(0.2)(x)  # Add dropout layer to prevent overfitting
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)  # Add dropout layer to prevent overfitting

output = Dense(64)(x)

model = Model(inputs=[input_layer, input_layer1, input_layer2, input_layer3], outputs=output)

# Compile the model
optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer=optimizer, loss=smape)

X_train = X_train.astype('float32')
y_train = y_train.astype('float32')
X_test = X_test.astype('float32')
y_test = y_test.astype('float32')

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# Fit the model with early stopping callback
model.fit([X_train, access_train, prj_train, spider_train], y_train, 
          validation_data=([X_test, access_test, prj_test, spider_test], y_test), 
          epochs=500, batch_size=64, callbacks=[early_stopping])

Epoch 1/500
1563/1563 [==============================] - 60s 33ms/step - loss: 24.3380 - val_loss: 19.9432
Epoch 2/500
1563/1563 [==============================] - 51s 33ms/step - loss: 20.1659 - val_loss: 19.7303
Epoch 3/500
1563/1563 [==============================] - 51s 32ms/step - loss: 19.9300 - val_loss: 19.5474
Epoch 4/500
1563/1563 [==============================] - 51s 33ms/step - loss: 19.8148 - val_loss: 19.3270
Epoch 5/500
1563/1563 [==============================] - 51s 32ms/step - loss: 19.6360 - val_loss: 19.3975
Epoch 6/500
1563/1563 [==============================] - 51s 33ms/step - loss: 19.5008 - val_loss: 18.8820
Epoch 7/500
1563/1563 [==============================] - 51s 32ms/step - loss: 19.4864 - val_loss: 19.1379
Epoch 8/500
1563/1563 [==============================] - 51s 32ms/step - loss: 19.6301 - val_loss: 18.9342
Epoch 9/500
1563/1563 [==============================] - 51s 32ms/step - loss: 19.7146 - val_loss: 19.1348
Epoch 10/500
1563/1563 [=============

# Submission

In [16]:
x_pred=[]
project_pred=[]
access_pred=[]
spider_pred=[]
projects = ['zh.wikipedia.org', 'fr.wikipedia.org', 'en.wikipedia.org', 'commons.wikimedia.org', 'ru.wikipedia.org', 'www.mediawiki.org', 'de.wikipedia.org', 'ja.wikipedia.org', 'es.wikipedia.org']

for i in tnrange(data.shape[0]):
    k=data.iloc[i].values[-300:]
    x_pred.append(k)

    for p in projects:
        if p in Page[i]:
            project_pred.append(p)
            break

    if('spider' in Page[i]):
        spider_pred.append('spider')
    else:
        spider_pred.append('non-spider') 
 
    k=max([i.start() for i in re.finditer('org_',Page[i])])   
    if('all-access' in Page[i][k:]):
        access_pred.append('all_access')
    if('desktop' in Page[i][k:]):
        access_pred.append('desktop')
    if('mobile' in Page[i][k:]): 
        access_pred.append('mobile')

x_pred=np.array(x_pred).reshape(data.shape[0],300,1)

access_pred=access_enc.transform(access_pred).reshape(len(access_pred),1)
project_pred=prj_enc.transform(project_pred).reshape(len(project_pred),1)
spider_pred=spider_enc.transform(spider_pred).reshape(len(spider_pred),1)  
x_pred = x_pred.astype('float32')
access_pred = access_pred.astype('float32')
project_pred = project_pred.astype('float32')
spider_pred = spider_pred.astype('float32')

pred=model.predict([x_pred,access_pred,project_pred,spider_pred])

/tmp/ipykernel_27/2483910096.py:7: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  for i in tnrange(data.shape[0]):


  0%|          | 0/145063 [00:00<?, ?it/s]

4534/4534 [==============================] - 60s 13ms/step


In [17]:
key = pd.read_csv("/kaggle/input/web-traffic-time-series-forecasting/key_2.csv.zip", compression = "zip")
sample_sub = pd.read_csv("/kaggle/input/web-traffic-time-series-forecasting/sample_submission_2.csv.zip", compression = "zip")

In [18]:
ids = key['Page'].values

visits = {}

for i in tnrange(data.shape[0]):
    date = datetime.date(2017, 9, 13)
    
    # Loop over the range of days from September 13 to November 13 (62 days)
    for j in range(62):
        # Construct the name by appending the date to the page name
        name = Page[i] + '_' + str(date)
        
        visits[name] = pred[i][j]
        
        # Increment the date by one day
        date += datetime.timedelta(days=1)


/tmp/ipykernel_27/3281572567.py:8: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  for i in tnrange(data.shape[0]):


  0%|          | 0/145063 [00:00<?, ?it/s]

In [19]:
for i in tnrange(len(ids)):
    sample_sub.at[i,'Visits']=visits[ids[i]]

/tmp/ipykernel_27/1157927782.py:1: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  for i in tnrange(len(ids)):


  0%|          | 0/8993906 [00:00<?, ?it/s]

/tmp/ipykernel_27/1157927782.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '505.3911437988281' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  sample_sub.at[i,'Visits']=visits[ids[i]]


In [20]:
sample_sub.to_csv('/kaggle/working/lstm_submit.csv',index=False)

In [21]:
sample_sub.shape

(8993906, 2)